# Development notebook for visualizing inferred bathymetries

by Jessica Scheick

Workflow based on previous methods and code developed by JScheick for Scheick et al 2019 *Remote Sensing*.

***Important note about CRS handling*** This code was developed while also learning about Xarray, rioxarray, rasterio, and other Python geospatial libraries. Since projections are not yet fully handled [smoothly] in any of those resources, and especially not integrated, there's little to no built in checking or handling of CRS. Instead, handling is done manually throughout the code and external to this notebook. This is critical to know because the CRS displayed by a rioxarray dataset may be from one variable added to the dataset, but is not necessarily the original (or read in) CRS for each variable in the dataset (hence the manual, external handling). The `get_mask` and `get_new_var_from_file` methods should reproject new data sources before adding them to the dataset.

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
import hvplot.xarray
# import hvplot.pandas
import holoviews as hv
hv.extension('bokeh','matplotlib')
from holoviews import dim, opts
import datetime as dt
import os
import panel as pn
pn.extension()
import pyproj
import rioxarray

In [ ]:
%load_ext autoreload
import icebath as icebath
from icebath.core import build_xrds
from icebath.utils import raster_ops as raster_ops
from icebath.utils import vector_ops as vector_ops
from icebath.core import fl_ice_calcs as icalcs
from icebath.core import build_gdf
%autoreload 2

## Read in inferred water depths

In [ ]:
outdir = "/Users/jessica/projects/bathymetry_from_bergs/results/"
resfile = [f for f in os.listdir(outdir) if f.endswith('_combined_results.gpkg')][0]
print(resfile)

In [ ]:
gdf = gpd.read_file(outdir+resfile, ignore_index=True)

In [ ]:
gdf.head() #[gdf['date'].dt.year.astype(int)==2016]

## Plot the measured and inferred values
Plots the gridded versus iceberg-freeboard-inferred values for all icebergs relative to the values in BedMachine and IBCAO.

Left plot shows measured values within the gridded datasets; right plot show the modeled/inferred values within the gridded data products (hence the larger error bars).

In [ ]:
from icebath.utils import plot as ibplot

In [ ]:
ibplot.meas_vs_infer_fig(gdf, save=False)

## Construct a contour map utilizing the results and gridded product data

### Create an Xarray dataset with the gridded products (BedMachine, IBCAO)

***Improvement: add CRS handling/checks to catch when a measurement dataset is incompatible and needs to be reprojected***

#### BedMachine Greenland

In [ ]:
measfile='/Users/jessica/mapping/datasets/160281892/BedMachineGreenland-2017-09-20.nc'
# measfile='/Users/jessica/mapping/datasets/160281892/BedMachineGreenland-2017-09-20_3413_'+ds.attrs['fjord']+'.nc'
# measfile='/home/jovyan/icebath/notebooks/supporting_docs/160281892/BedMachineGreenland-2017-09-20.nc'
# measfile='/home/jovyan/icebath/notebooks/supporting_docs/160281892/BedMachineGreenland-2017-09-20_'+ds.attrs['fjord']+'.nc'

#### IBCAOv4
https://www.gebco.net/data_and_products/gridded_bathymetry_data/arctic_ocean/

Source keys: https://www.gebco.net/data_and_products/gridded_bathymetry_data/gebco_2020/

Downloaded Feb 2021

**NOTE** IBCAO has it's own Polar Stereo projection (EPSG:3996: WGS 84/IBCAO Polar Stereographic) so it needs to be reprojected before being applied to these datasets.
See: https://spatialreference.org/ref/?search=Polar+Stereographic

The variable name of one of the IBCAO input files must be changed during reprojection or it results in an error during merging.

In [ ]:
measfile2a='/Users/jessica/mapping/datasets/IBCAO_v4_200m_ice_3413.nc'
# measfile2a='/Users/jessica/mapping/datasets/IBCAO_v4_200m_ice_3413_'+ds.attrs['fjord']+'.nc'
# measfile2a='/home/jovyan/icebath/notebooks/supporting_docs/IBCAO_v4_200m_ice_3413.nc'
# measfile2a='/home/jovyan/icebath/notebooks/supporting_docs/IBCAO_v4_200m_ice_3413_'+ds.attrs['fjord']+'.nc'
measfile2b='/Users/jessica/mapping/datasets/IBCAO_v4_200m_TID_3413.nc'
# measfile2b='/home/jovyan/icebath/notebooks/supporting_docs/IBCAO_v4_200m_TID_3413.nc'

#### Get the coordinates of BedMachine nearest the regional bounds

In [ ]:
from icebath.core import berggdf as bgdf
# Kane region bbox
kanereg = (-552082, -1125990, -248996, -828936)
# bbox = (-552125, -1126025, -248975, -828875)
bbox = bgdf.get_needed_extent(measfile, kanereg)

#### Create the dataset

In [ ]:
resds = build_xrds.read_netcdfs([measfile, measfile2a, measfile2b], extent=bbox)
resds = resds.squeeze(drop=True)
resds

### Add the iceberg-inferred water depths

In [ ]:
gdf['centroid'] = gdf.geometry.centroid
gdf['poly'] = gdf.geometry
gdf = gdf.drop(columns='geometry')
gdf.set_geometry(col="centroid", inplace=True)
gdf['geometry'] = gdf.centroid

In [ ]:
from functools import partial
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial, rasterize_image
from shapely.geometry import Point

Note: make_geocube does not recognize geometry columns not named "geometry". It also only works for point geometries [when rasterizing?], not polygons (hence the use of the centroid, computed above in the geodataframe).

There are multiple options for gridding the data, given by rasterize_function.

In [ ]:
import rasterio
gdf_grid = make_geocube(vector_data=gdf,
                        measurements=["filtered_draft_med", "filtered_draft_err"],
                        like=resds,
                        fill=np.nan,
#                         rasterize_function=partial(rasterize_points_griddata, method="linear", filter_nan=True)
                        rasterize_function=partial(rasterize_image, filter_nan=True)
                       )

In [ ]:
gdf_grid.filtered_draft_med.plot()

In [ ]:
gdf_grid.filtered_draft_med.plot()

In [ ]:
resds["filtered_draft_med"] = gdf_grid.filtered_draft_med
resds["filtered_draft_err"] = gdf_grid.filtered_draft_err
del gdf_grid
resds

In [ ]:
resds.mask.plot()

In [ ]:
resds.filtered_draft_err.plot()

In [ ]:
scrolldem = resds['filtered_draft_err'].hvplot.image(x='x', y='y',datashade=False, rasterize=True, aspect='equal', cmap='magma', dynamic=True,
                       xlabel="x (km)", ylabel="y (km)", colorbar=True) #turn off datashade to see hover values + colorbar
scrolldem

Notes:
There are no measurements for much of Kane (in either product)

### Upsample the data and create the contour plot

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
%matplotlib inline

In [ ]:
resds

In [ ]:
# testing code
# all_bathy = resds.filtered_draft_med_rast.where(~np.isnan(resds.filtered_draft_med_rast), other=meas_only)
# all_bathy.hvplot()

In [ ]:
# POTENTIAL NEXT STEPS:
# try doing this in holoviews to see if it is faster (especially to layer the plots) --> not at first attempt
# fix the 0/nodata issue so that it uses the available data to contour where possible (but doesn't contour to 0)
# move this plot creation into plot.py and just run it here to get the output

In [ ]:
# generate data for plots - measured and inferred bathymetry
# NOTE: this uses IBCAO because it has the OMG data included, whereas BedMachine v3 does not
interp_meth = "nearest" # with rio.interpolate_na, you can only use nearest right now
# linear and cubic need a fill_value specified, which is ignored with nearest but ends up being passed to
# scipy as "None", thus breaking in scipy.interpolate.ndgriddata.griddata (perhaps in the def fill_value of interpolate.py?).
# However, changing meas_only.rio._nodata doesn't help, and the inputs seem to be fine (plus they work for "nearest")
coarse = 8

# measurements only
obs = -resds.ibcao_bathy.where(resds.z<40).combine_first(-resds.bed.where(resds.source>=10))
meas_only = obs.where(resds.mask==0, other=0)

# inferred points + measured points
all_bathy = resds.filtered_draft_med.where(~np.isnan(resds.filtered_draft_med), 
                                           other=meas_only)

# coarsen and interpolate
# meas_only = meas_only.coarsen(x=coarse, boundary='pad').mean().coarsen(y=coarse, boundary="pad").mean()
meas_only = meas_only.coarsen(x=coarse, y=coarse, boundary='pad').mean()
# meas_only.rio._nodata = np.nan
# meas_only = meas_only.rio.interpolate_na(method=interp_meth)
# all_bathy = all_bathy.coarsen(x=coarse, boundary='pad').mean().coarsen(y=coarse, boundary="pad").mean()
all_bathy = all_bathy.coarsen(x=coarse, y=coarse, boundary='pad').mean()
# all_bathy = all_bathy.rio.interpolate_na(method=interp_meth)

# non-shared plot info
plot_title = 'Kane Basin Bathymetry'
subplot_title = 'sonar measured'

# output filename
fig_name = "bathymetry.png"

#define range of colors for plotting
v=np.linspace(0,600,13)

In [ ]:
# generate data for plots - uncertainty on bathymetry
# NOTE: this used BedMachine, because IBCAO doesn't include uncertainties
interp_meth = "nearest" # with rio.interpolate_na, you can only use nearest right now
# linear and cubic need a fill_value specified, which is ignored with nearest but ends up being passed to
# scipy as "None", thus breaking in scipy.interpolate.ndgriddata.griddata (perhaps in the def fill_value of interpolate.py?).
# However, changing meas_only.rio._nodata doesn't help, and the inputs seem to be fine (plus they work for "nearest")
coarse = 8

# measurements only
# obs = -resds.errbed.where(resds.z<40).combine_first(-resds.bed.where(resds.source>=10))
obs = resds.errbed
meas_only = obs.where(resds.mask==0, other=0)

# inferred points + measured points
all_bathy = resds.filtered_draft_err.where(~np.isnan(resds.filtered_draft_err), 
                                           other=meas_only)

# coarsen and interpolate
# meas_only = meas_only.coarsen(x=coarse, boundary='pad').mean().coarsen(y=coarse, boundary="pad").mean()
meas_only = meas_only.coarsen(x=coarse, y=coarse, boundary='pad').mean()
# meas_only.rio._nodata = np.nan
# meas_only = meas_only.rio.interpolate_na(method=interp_meth)
# all_bathy = all_bathy.coarsen(x=coarse, boundary='pad').mean().coarsen(y=coarse, boundary="pad").mean()
all_bathy = all_bathy.coarsen(x=coarse, y=coarse, boundary='pad').mean()
# all_bathy = all_bathy.rio.interpolate_na(method=interp_meth)

# non-shared plot info
plot_title = 'Kane Basin Bathymetry Uncertainty'
subplot_title = 'BedMachine'

# output filename
fig_name = "bathy_uncertainty.png"

#define range of colors for plotting
v=np.linspace(0,300,11)


# redefine some variables to make a test plot with the difference in the uncertainties
new_all = meas_only - all_bathy
meas_only = all_bathy
all_bathy = new_all
v = np.linspace(0,100, 11)

In [ ]:
# Note: contouring excludes masked or nan regions (see matplotlib contourf_demo) - an issue for Kane's sparse observations
# the pyinterp library (https://pangeo-pyinterp.readthedocs.io/en/stable/index.html) could provide some help here

loc = ['Kane Basin']
letters=['a','b']

# plt.close(fig)

fig, axes = plt.subplots(1,2, figsize=(12,5)) #8,3

#define range of colors for plotting
# v=np.linspace(0,600,13)  # defined above per-plot
bathy_cmap = LinearSegmentedColormap.from_list('bathy_cmap', [(0.85,0.9,0.98),(0,0,.5)], 12)   #plt.cm.Blues

# left plot (contours from measurements only)
CS = xr.plot.contour(meas_only, x='x', y='y', ax=axes[0], linewidths=0.3, colors=[(0.95,0.95,0.95)], add_labels=False)
CS = xr.plot.contourf(meas_only, x='x', y='y', ax=axes[0], levels=v, cmap=bathy_cmap, add_labels=False)

# right plot: contours from inferred points and measured points
# Note: currently this uses the iceberg-inferred water depth for the entire area of the iceberg, rather than a single point
CS2 = xr.plot.contour(all_bathy, x='x', y='y', ax=axes[1], linewidths=0.3, colors=[(0.95,0.95,0.95)], add_labels=False)
CS2 = xr.plot.contourf(all_bathy, x='x', y='y', ax=axes[1], levels=v, cmap=bathy_cmap, add_labels=False)

# add land and measured bathymetry extent
# plot formatting
landcmap = LinearSegmentedColormap.from_list('land_cmap', [(0.65,0.45,0.35,1),(1,1,1,0)], 2)

xmin, xmax = -500000, -350000
ymin, ymax = -1120000, -970000

for n in range(0,2):
    land = resds.mask.where(resds.mask == 0).fillna(-888).plot.imshow(x='x', y='y', ax=axes[n], cmap=landcmap, zorder=30, add_labels=False)
    land.colorbar.remove()
#     xr.plot.contour(ds.bmach_source, x='x', y='y', ax=axes[n], levels=[0,10,100], linewidths=1, colors='black', add_labels=False)
    
    axes[n].axis('equal')
    axes[n].set_ylim(ymin,ymax)
    axes[n].set_xlim(xmin,xmax)
    
    plt.setp(axes[n].get_xticklabels(), rotation=30, horizontalalignment='right')

# update plot labels, etc.
CS.colorbar.remove()
CS2.colorbar.remove()
cbaxes = fig.add_axes([0.95, 0.1, 0.02, 0.75])
plt.colorbar(CS2, cax=cbaxes)
cbaxes.set_title('meters', fontsize=12)


#turn off y axis labels
axes[1].yaxis.set_ticklabels([])

#label each plot
axes[0].set_title(subplot_title, fontsize=11)
axes[0].text(0.02, 0.95, letters[0], weight='bold', transform=axes[0].transAxes, zorder=40)
axes[1].set_title(subplot_title + ' + iceberg-inferred', fontsize=11)
axes[1].text(0.02, 0.95, letters[1], weight='bold', transform=axes[1].transAxes, zorder=40)

fig.text(0.5, 0.03,'Easting (m)', ha='center', va='center', fontsize=12)
fig.text(0.03, 0.5,'Northing (m)', ha='center', va='center', rotation='vertical', fontsize=12)

plt.suptitle(plot_title, fontsize=14)
fig.subplots_adjust(hspace=0.3, wspace = 0.14, top=0.87, left=0.1, right=0.90, bottom=0.17)


In [ ]:
fig_save_path = outdir[:-8] + "project_report(2021)/figures/"
fig.savefig(fig_save_path+fig_name, format='png', dpi=1200)

In [ ]:
# compute some stats on the uncertainty improvements for the calcs at the end of the uncertainty maps data prep section
red_uncert = all_bathy.values
red_uncert = red_uncert[red_uncert>0]
print(min(red_uncert))
print(max(red_uncert))
print(np.mean(red_uncert))
print(np.median(red_uncert))

In [ ]:
plt.hist(red_uncert)

## Visualize the range of elevation (area) values for a single iceberg (DEM)

In [ ]:
vals = gdf[gdf['date'].dt.year.astype(int)==2016].geometry.area

In [ ]:
axHist = plt.subplot()

_ = axHist.hist(vals, np.arange(50000,300000,25000), color="silver")

axHist.set_xlabel('Elevation (m)')
axHist.set_ylabel('Count')
axHist.set_title('DEM Pixel Elevations')

plt.show()